In [33]:
import numpy as np
import pandas as pd

from scipy.interpolate import griddata
from scipy.special import gamma
from scipy.stats import percentileofscore
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF

In [34]:
np.random.seed(0)

obj = "liao_test"

m = 5
n = 10000

percentile = 50

iterations = 10

laplace_alpha = 0.1

In [35]:
def liao_physical(X):
    alpha = 4 + 3 * X[2]**2 - 2 * X[3]**2 + 3 * X[2] * X[3]
    beta = 100 + 10 * X[0]**2 + 6 * X[1]**2 - 5 * X[0] * X[1] + 10 * X[2]**1.5 - 8 * X[3]**2

    return beta * gamma(1 + 1 / alpha)

def liao_simulation(X):
    alpha = 4.5 + 2.5 * X[2]**2 - 2.5 * X[3]**2 + 3.5 * X[2] * X[3]
    # beta = 100 + 9 * X[0]**2 + 6.5 * X[1]**2 + 10 * X[2]**2 - 8 * X[3]**2
    beta = 100 + 9 * 1**2 + 6.5 * 1**2 + 10 * X[2]**2 - 8 * X[3]**2

    return beta * gamma(1 + 1 / alpha)

objectives = {
    "liao_test": {
        "func": liao_physical,
        "bounds": [(0, 1)] * 4
    }
}

In [36]:
obj = list(objectives.keys())[-1] if obj is None else obj

x_range = objectives[obj]["bounds"]

X = [np.arange(*x_range[i], ((x_range[i][1] - x_range[i][0]) / 100)) for i in range(len(x_range))]
X = np.meshgrid(*X)

Y = objectives[obj]["func"](X)

y_range = (np.floor(np.min(Y)) - 1, np.ceil(np.max(Y)) + 1)

In [37]:
ran_err = lambda n, x: np.random.normal(0, x, n)

In [38]:
df = pd.DataFrame(np.random.randint(100, size=(m, len(x_range))), columns=[f"i_{i}" for i in range(len(x_range))])

for i in range(len(x_range)):
    df[f"x_{i}"] = X[0][0, df[f"i_{i}"], 0, 0]
    
df["y"] = Y[tuple([df[f"i_{i}"] for i in range(len(x_range))])] + ran_err(m, 10)

for idx in range(1, iterations + 1):
    krnl = RBF(length_scale=1)
    model = GaussianProcessRegressor(kernel=krnl, normalize_y=False, random_state=3, alpha=0.001)

    tmp_df = pd.DataFrame(np.random.randint(100, size=(n, len(x_range))), columns=[f"i_{i}" for i in range(len(x_range))])

    for i in range(len(x_range)):
        tmp_df[f"x_{i}"] = X[0][0, tmp_df[f"i_{i}"], 0, 0]
        
    tmp_df["y"] = liao_simulation([tmp_df[f"x_{i}"] for i in range(len(x_range))]) + ran_err(n, 10)

    model.fit(tmp_df[[f"x_{i}" for i in range(len(x_range))]], tmp_df["y"])

    pred = model.predict(tmp_df[[f"x_{i}" for i in range(len(x_range))]])

    y = griddata(tuple([tmp_df[f"x_{i}"] for i in range(len(x_range))]), pred, tuple(X), method="linear", fill_value=y_range[0])

    tmp_df = pd.DataFrame(np.random.randint(0, 100, size=(10000, 4)), columns=[f"i_{i}" for i in range(len(x_range))])

    for i in range(len(x_range)):
        tmp_df[f"x_{i}"] = X[0][0, tmp_df[f"i_{i}"], 0, 0]
        
    tmp_df["y"] = liao_simulation([tmp_df[f"x_{i}"] for i in range(len(x_range))])

    mag = y[tuple([tmp_df[f"i_{i}"] for i in range(len(x_range))])]

    p = mag - y.min() + laplace_alpha

    p = np.where(p > np.percentile(p, percentile), p, laplace_alpha)

    p /= p.sum()

    tmp_df = tmp_df.loc[np.random.choice(tmp_df.index, size=m, p=p, replace=False)]

    df = pd.concat([df, tmp_df], ignore_index=True)

krnl = RBF(length_scale=1)
model = GaussianProcessRegressor(kernel=krnl, normalize_y=False, random_state=3, alpha=0.001)

model.fit(df[[f"x_{i}" for i in range(len(x_range))]], df["y"])

c:\Program Files\Python310\lib\site-packages\sklearn\gaussian_process\kernels.py:420: ConvergenceWarning: The optimal value found for dimension 0 of parameter length_scale is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(
c:\Program Files\Python310\lib\site-packages\sklearn\gaussian_process\kernels.py:420: ConvergenceWarning: The optimal value found for dimension 0 of parameter length_scale is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(
c:\Program Files\Python310\lib\site-packages\sklearn\gaussian_process\kernels.py:420: ConvergenceWarning: The optimal value found for dimension 0 of parameter length_scale is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(
c:\Program Files\Python310\lib\site-packages\sklearn\gaussian_process\kernels.py:420: Converge

GaussianProcessRegressor(alpha=0.001, kernel=RBF(length_scale=1),
                         random_state=3)

In [39]:
pred = model.predict(df[[f"x_{i}" for i in range(len(x_range))]])

y = griddata(tuple([df[f"x_{i}"] for i in range(len(x_range))]), pred, tuple(X), method="linear", fill_value=y_range[0])

print("Predicted maximum:", Y[np.unravel_index(y.argmax(), y.shape)], "at", np.unravel_index(y.argmax(), y.shape))
print("Actual maximum:", Y.max(), "at", np.unravel_index(Y.argmax(), Y.shape))

df = pd.DataFrame(np.random.randint(0, 100, size=(10000, 4)), columns=[f"i_{i}" for i in range(len(x_range))])

for i in range(len(x_range)):
    df[f"x_{i}"] = X[0][0, df[f"i_{i}"], 0, 0]
    
df["y"] = Y[tuple([df[f"i_{i}"] for i in range(len(x_range))])]

pred = model.predict(df[[f"x_{i}" for i in range(len(x_range))]])

sum = np.sum((pred - df["y"])**2)
rmse = np.sqrt(sum / len(df["y"]))

print("RMSE:", rmse)

Predicted maximum: 105.94955761342028 at (38, 60, 99, 8)
Actual maximum: 112.95467083064564 at (99, 99, 99, 13)
RMSE: 17.645290520700872


In [40]:
print(np.max(Y) - np.min(Y))
print(np.min(Y))
print(np.max(Y))
print(np.percentile(Y, 25))
print(np.median(Y))
print(np.average(Y))
print(np.percentile(Y, 75))
print(percentileofscore(Y.flatten(), Y[np.unravel_index(y.argmax(), y.shape)]))

31.311183072263816
81.64348775838182
112.95467083064564
92.86990236199593
96.55343347726301
96.65730484672557
100.47367416188011
95.922811
